# Backtesting with backtrader library
Notes:  
Timestamp must be in the exact format specified for datafeed   **Timestamp needs to be first column index**  
Params for datafeed need to be specified as tuple of tuples  
Within strategy data is accessed via self.datas[0] for ex: self.datas[0].high_delta[0] would get you the first timeperiod's high_delta  
The current data has already happened and cannot be used to execute an order (ex. you cannot look at the current close price and simultaneously buy the current close, the order will be executed at open instead), orders will be executed on the following day.

## Import Data and Run Backtest

In [1]:
import os
import sys
sys.path.append('../scripts')
from backtesting_backtrader import *

In [2]:
def capstone_data_prep(pred_folder_name, pred_col_name, ticker_df_filepath, starting_cash=100000, dirty_data=False):
    """Main loop, preps data and executes backtrader on all stocks in the prediction folder"""
    ticker_backtesting_dict = {}
    for pred_file in os.listdir(f'../data/ticker_predictions/{pred_folder_name}'):
        # collect data
        ticker_name = pred_file.split('_')[0]
        pred_df = pd.read_csv(f'../data/ticker_predictions/{pred_folder_name}/{pred_file}')
        ticker_df = pd.read_csv(ticker_df_filepath + f'/{ticker_name}_full_data.csv')
        
        # make timestamp in column position 0, SPECIFIC TO FULL DATA IN FILE 
        ticker_df = ticker_df.rename({'reportperiod':'timestamp'}, axis=1)
        ticker_cols = list(ticker_df.columns)
        ticker_cols[5] = 'ts_cpy'
        ticker_cols[0] = 'timestamp'
        ticker_df.columns = ticker_cols
        ticker_df['timestamp'] = ticker_df['ts_cpy']
        # hack
        if dirty_data == False:
            pred_df['timestamp'] = pd.to_datetime(pred_df['timestamp'])
            ticker_df['timestamp'] = pd.to_datetime(ticker_df['timestamp'])
        # merge prediction data with full data, on key = timestamp
        ticker_df = ticker_df.merge(pred_df.loc[:, ['timestamp', pred_col_name]], on='timestamp')
        # new prediction column name = "prediction", important because backtester strategy looks for this column name
        ticker_df = ticker_df.rename({pred_col_name:'prediction'}, axis=1)
        # remove all non-prediction rows in prediction field
        ticker_df = ticker_df[ticker_df.prediction > 0]
        # skip nonfeatures, keep timestamp, SPECIFIC TO FULL DATA IN FILE 
        nonfeatures = ticker_df.columns[1:7]
        features = [col for col in ticker_df.columns if col not in nonfeatures]
        # get index of start and end dates for trading
        ticker_df = ticker_df[features]
        ticker_df.columns = [c.lower() for c in ticker_df.columns]
        idx1 = ticker_df.prediction[ticker_df.prediction > 0].index[0]
        idx2 = ticker_df.prediction[ticker_df.prediction > 0].index[-1]
        start_date = pred_df.timestamp[idx1]
        end_date  = pred_df.timestamp[idx2]
        
        # Prepare data for backtrader
        prep_data = prepare_data(ticker_df.copy(), start_date, end_date, 'temp/prep_data.csv')
        class DataFeed(bt.feeds.GenericCSVData):
            lines = tuple(prep_data[2])
            params = prep_data[1]
        
        print(f"--------Ticker Name-------: {ticker_name}")
        strategies = [BuyAndHold, BasicStrategy]
        ticker_backtesting_dict[ticker_name] = {BuyAndHold.__name__:{},
                                                BasicStrategy.__name__:{}}
        for i in range(len(strategies)):
            # initialize everything and run strategy
            cerebro = bt.Cerebro(cheat_on_open=True)
            cerebro.broker.setcash(starting_cash)
            data = DataFeed(dataname=prep_data[0])
            cerebro.adddata(data)
            cerebro.addstrategy(strategies[i])
            cerebro.addanalyzer(bt.analyzers.SharpeRatio, timeframe=bt.TimeFrame.Months, riskfreerate=0, _name = 'sharpe')
            cerebro.addanalyzer(bt.analyzers.DrawDown, _name = 'dd')
            back = cerebro.run()
            total_return = cerebro.broker.getvalue() / starting_cash - 1
            
            ticker_backtesting_dict[ticker_name][strategies[i].__name__] = {'total_return': total_return, 
                                                                         'monthly_sharpe': back[0].analyzers.sharpe.get_analysis(),
                                                                         'drawdown': back[0].analyzers.dd.get_analysis(),
                                                                         'cerebro_object': cerebro
                                                                        }
                                                     
        
    return ticker_backtesting_dict

### XGBoost Results

In [3]:
xgb_backtesting_dict = capstone_data_prep('xgboost_preds', 'holdout_preds', '../data/ticker_data', dirty_data=True)

avg_analysis_dict_xgb = {'BuyAndHold':
                     {'return_avg':[], 'sharpe_avg':[], 'max_drawdown_avg':[]},
                     'BasicStrategy':
                     {'return_avg':[], 'sharpe_avg':[], 'max_drawdown_avg':[]} }
# get values
for ticker, ticker_dict in xgb_backtesting_dict.items():
    for strat_name in ['BuyAndHold', 'BasicStrategy']:
        for analysis_type, value in ticker_dict[strat_name].items():
            if analysis_type == 'drawdown':
                 avg_analysis_dict_xgb[strat_name]['max_drawdown_avg'].append(value['max']['drawdown'])
            elif analysis_type == 'monthly_sharpe':
                avg_analysis_dict_xgb[strat_name]['sharpe_avg'].append(value['sharperatio'])
            elif analysis_type == 'total_return':
                avg_analysis_dict_xgb[strat_name]['return_avg'].append(value)

# calculate averages
for strat, val_dict in avg_analysis_dict_xgb.items():
    for val_type, val_list in avg_analysis_dict_xgb[strat].items():
        el_sum = 0
        for el in val_list:
            el_sum+=el
        avg_analysis_dict_xgb[strat][val_type] = el_sum / len(val_list)

--------Ticker Name-------: ADSK
2019-09-19, BUY EXECUTED, 151.39
Stop price: 329.07000732421875
ROI:        117.27%
***The limit sells for this strategy are valid for 1 day***
2019-09-18, Current Portfolio Value : 100000.0
2019-09-18, MARKET BUY CREATE 660 shares at next open, current close price: 151.11000061035156
2019-09-19, BUY EXECUTED, 151.39
2019-09-19, Current Portfolio Value : 101313.40362548828
2019-09-19, MARKET SELL CREATE. PREDICTION < CURRENT CLOSE
2019-09-20, SELL EXECUTED, 153.14
2019-09-20, Current Portfolio Value : 101155.0
2019-09-20, MARKET BUY CREATE 663 shares at next open, current close price: 153.33999633789062
2019-09-23, BUY EXECUTED, 152.46
2019-09-23, Current Portfolio Value : 101095.32231140137
2019-09-23, LIMIT SELL CREATE 663 shares at 155.39136422503358
2019-09-24, Current Portfolio Value : 98947.20878601074
2019-09-24, LIMIT SELL CREATE 663 shares at 154.40838642025483
2019-09-25, Current Portfolio Value : 97820.10069274902
2019-09-25, LIMIT SELL CREAT

2020-08-19, BUY EXECUTED, 244.40
2020-08-19, Current Portfolio Value : 230078.9561199773
2020-08-19, LIMIT SELL CREATE 953 shares at 245.9131653883451
2020-08-20, SELL EXECUTED, 245.91
2020-08-20, Current Portfolio Value : 234360.9444800653
2020-08-20, MARKET BUY CREATE 939 shares at next open, current close price: 249.1499938964844
2020-08-21, BUY EXECUTED, 249.33
2020-08-21, Current Portfolio Value : 233365.60677254573
2020-08-21, LIMIT SELL CREATE 939 shares at 252.4164794896533
2020-08-24, Current Portfolio Value : 236680.27562630552
2020-08-24, MARKET SELL CREATE. PREDICTION < CURRENT CLOSE
2020-08-25, SELL EXECUTED, 249.02
2020-08-25, Current Portfolio Value : 234069.85677254573
2020-08-25, MARKET BUY CREATE 948 shares at next open, current close price: 252.2400054931641
2020-08-26, BUY EXECUTED, 246.66
2020-08-26, Current Portfolio Value : 235539.25966561213
2020-08-26, LIMIT SELL CREATE 948 shares at 255.54700274039902
2020-08-27, Current Portfolio Value : 230704.45387947935
20

2021-07-09, Current Portfolio Value : 255489.00207130195
2021-07-09, LIMIT SELL CREATE 864 shares at 298.4093928905713
2021-07-12, Current Portfolio Value : 252508.19152442695
2021-07-12, LIMIT SELL CREATE 864 shares at 299.33157049512374
2021-07-13, Current Portfolio Value : 256387.56945411445
2021-07-13, MARKET SELL CREATE. PREDICTION < CURRENT CLOSE
2021-07-14, SELL EXECUTED, 298.50
2021-07-14, Current Portfolio Value : 258184.67785255195
2021-07-14, MARKET BUY CREATE 867 shares at next open, current close price: 297.1000061035156
2021-07-15, BUY EXECUTED, 297.48
2021-07-15, Current Portfolio Value : 255852.44573585273
2021-07-15, LIMIT SELL CREATE 867 shares at 301.01756022872
2021-07-16, Current Portfolio Value : 254586.60668555976
2021-07-16, LIMIT SELL CREATE 867 shares at 298.7336812423382
2021-07-19, Current Portfolio Value : 249367.27621070624
2021-07-19, LIMIT SELL CREATE 867 shares at 297.25412723459164
2021-07-20, SELL EXECUTED, 297.25
2021-07-20, Current Portfolio Value :

--------Ticker Name-------: HD
2019-09-19, BUY EXECUTED, 231.15
Stop price: 368.5899963378906
ROI:        59.37%
***The limit sells for this strategy are valid for 1 day***
2019-09-18, Current Portfolio Value : 100000.0
2019-09-18, MARKET BUY CREATE 432 shares at next open, current close price: 230.8300018310547
2019-09-19, BUY EXECUTED, 231.15
2019-09-19, Current Portfolio Value : 98691.04052734374
2019-09-19, LIMIT SELL CREATE 432 shares at 232.73308312018344
2019-09-20, Current Portfolio Value : 97200.64184570311
2019-09-20, LIMIT SELL CREATE 432 shares at 230.00073371676262
2019-09-23, Current Portfolio Value : 97883.20263671874
2019-09-23, LIMIT SELL CREATE 432 shares at 226.52229321877533
2019-09-24, SELL EXECUTED, 228.14
2019-09-24, Current Portfolio Value : 98699.68237304686
2019-09-24, MARKET BUY CREATE 435 shares at next open, current close price: 225.41000366210938
2019-09-25, BUY EXECUTED, 226.59
2019-09-25, Current Portfolio Value : 99356.53662109372
2019-09-25, MARKET SEL

2020-08-25, BUY EXECUTED, 287.27
2020-08-25, Current Portfolio Value : 171415.72487151902
2020-08-25, LIMIT SELL CREATE 599 shares at 289.114114522934
2020-08-26, SELL EXECUTED, 289.11
2020-08-26, Current Portfolio Value : 173203.2065459518
2020-08-26, MARKET BUY CREATE 592 shares at next open, current close price: 291.92999267578125
2020-08-27, BUY EXECUTED, 292.22
2020-08-27, Current Portfolio Value : 171077.92871392056
2020-08-27, LIMIT SELL CREATE 592 shares at 294.336813723261
2020-08-28, Current Portfolio Value : 169692.6508818893
2020-08-28, LIMIT SELL CREATE 592 shares at 291.00961913319765
2020-08-31, Current Portfolio Value : 168952.6508818893
2020-08-31, LIMIT SELL CREATE 592 shares at 288.6503306616505
2020-09-01, Current Portfolio Value : 169485.44726860806
2020-09-01, LIMIT SELL CREATE 592 shares at 287.3900250185688
2020-09-02, SELL EXECUTED, 287.39
2020-09-02, Current Portfolio Value : 170343.86063428828
2020-09-02, MARKET BUY CREATE 592 shares at next open, current clo

...

2020-07-29, BUY EXECUTED, 224.19
2020-07-29, Current Portfolio Value : 98211.59438776644
2020-07-29, LIMIT SELL CREATE 453 shares at 225.6385031357986
2020-07-30, Current Portfolio Value : 96508.30996393834
2020-07-30, LIMIT SELL CREATE 453 shares at 218.60149654204173
2020-07-31, Current Portfolio Value : 96684.97968744909
2020-07-31, LIMIT SELL CREATE 453 shares at 214.805340318324
2020-08-03, Current Portfolio Value : 96639.68383478794
2020-08-03, LIMIT SELL CREATE 453 shares at 215.1990889260524
2020-08-04, Current Portfolio Value : 95303.33521723421
2020-08-04, LIMIT SELL CREATE 453 shares at 215.0981368298653
2020-08-05, Current Portfolio Value : 95792.56913447056
2020-08-05, LIMIT SELL CREATE 453 shares at 212.11978039648602
2020-08-06, SELL EXECUTED, 212.12
2020-08-06, Current Portfolio Value : 96218.29297194982
2020-08-06, MARKET BUY CREATE 456 shares at next open, current close price: 210.63999938964844
2020-08-07, BUY EXECUTED, 210.64
2020-08-07, Current Portfolio Value : 96

2021-06-16, BUY EXECUTED, 342.11
2021-06-16, Current Portfolio Value : 152330.24934155404
2021-06-16, LIMIT SELL CREATE 444 shares at 345.0858459968076
2021-06-17, SELL EXECUTED, 345.09
2021-06-17, Current Portfolio Value : 153225.2817121835
2021-06-17, MARKET BUY CREATE 444 shares at next open, current close price: 346.55999755859375
2021-06-18, BUY EXECUTED, 345.04
2021-06-18, Current Portfolio Value : 152190.75412429287
2021-06-18, LIMIT SELL CREATE 444 shares at 349.8916156152627
2021-06-21, Current Portfolio Value : 152070.87900222256
2021-06-21, LIMIT SELL CREATE 444 shares at 346.0354019611441
2021-06-22, Current Portfolio Value : 153482.79575026943
2021-06-22, LIMIT SELL CREATE 444 shares at 345.73201335869817
2021-06-23, SELL EXECUTED, 346.19
2021-06-23, Current Portfolio Value : 153735.87900222256
2021-06-23, MARKET BUY CREATE 441 shares at next open, current close price: 346.95001220703125
2021-06-24, BUY EXECUTED, 348.08
2021-06-24, Current Portfolio Value : 152192.37900222

--------Ticker Name-------: XOM
2019-09-17, BUY EXECUTED, 73.82
Stop price: 65.72000122070311
ROI:        -10.97%
***The limit sells for this strategy are valid for 1 day***
2019-09-16, Current Portfolio Value : 100000.0
2019-09-16, MARKET BUY CREATE 1354 shares at next open, current close price: 73.7300033569336
2019-09-17, BUY EXECUTED, 73.82
2019-09-17, Current Portfolio Value : 99119.89793395996
2019-09-17, LIMIT SELL CREATE 1354 shares at 74.29306357801579
2019-09-18, Current Portfolio Value : 98646.0
2019-09-18, LIMIT SELL CREATE 1354 shares at 73.72878175052392
2019-09-19, Current Portfolio Value : 97982.54289245605
2019-09-19, LIMIT SELL CREATE 1354 shares at 73.37611041312813
2019-09-20, Current Portfolio Value : 97644.04289245605
2019-09-20, LIMIT SELL CREATE 1354 shares at 72.88237054077398
2019-09-23, Current Portfolio Value : 97711.73669433594
2019-09-23, LIMIT SELL CREATE 1354 shares at 72.63046134439719
2019-09-24, Current Portfolio Value : 96371.27958679199
2019-09-24, 

2020-08-05, BUY EXECUTED, 44.15
2020-08-05, Current Portfolio Value : 78692.60024330516
2020-08-05, LIMIT SELL CREATE 1794 shares at 43.86431727443779
2020-08-06, SELL EXECUTED, 43.86
2020-08-06, Current Portfolio Value : 78718.2881710733
2020-08-06, MARKET BUY CREATE 1820 shares at next open, current close price: 43.63999938964844
2020-08-07, BUY EXECUTED, 43.23
2020-08-07, Current Portfolio Value : 79100.48650481353
2020-08-07, LIMIT SELL CREATE 1820 shares at 44.03585749549347
2020-08-10, Current Portfolio Value : 81047.88594939362
2020-08-10, MARKET SELL CREATE. PREDICTION < CURRENT CLOSE
2020-08-11, SELL EXECUTED, 45.50
2020-08-11, Current Portfolio Value : 82849.68900420319
2020-08-11, MARKET BUY CREATE 1863 shares at next open, current close price: 44.97000122070313
2020-08-12, BUY EXECUTED, 44.46
2020-08-12, Current Portfolio Value : 82160.38099410187
2020-08-12, LIMIT SELL CREATE 1863 shares at 45.37202683705619
2020-08-13, Current Portfolio Value : 80148.33758284699
2020-08-1

2021-06-16, LIMIT SELL CREATE 1916 shares at 64.93105418127925
2021-06-17, Current Portfolio Value : 118778.51212020352
2021-06-17, LIMIT SELL CREATE 1916 shares at 64.75391931169271
2021-06-18, Current Portfolio Value : 115732.07182784511
2021-06-18, LIMIT SELL CREATE 1916 shares at 62.7446242572546
2021-06-21, Current Portfolio Value : 119928.10919661954
2021-06-21, MARKET SELL CREATE. PREDICTION < CURRENT CLOSE
2021-06-22, SELL EXECUTED, 62.85
2021-06-22, Current Portfolio Value : 120426.26598067716
2021-06-22, MARKET BUY CREATE 1880 shares at next open, current close price: 63.790000915527344
2021-06-23, BUY EXECUTED, 64.05
2021-06-23, Current Portfolio Value : 120821.06425948575
2021-06-23, LIMIT SELL CREATE 1880 shares at 64.43288877694613
2021-06-24, SELL EXECUTED, 64.43
2021-06-24, Current Portfolio Value : 121146.09114403119
2021-06-24, MARKET BUY CREATE 1875 shares at next open, current close price: 64.51999664306639
2021-06-25, BUY EXECUTED, 64.61
2021-06-25, Current Portfol

### Baseline results

In [ ]:
baseline_results_avg = {}
baseline_results_full = {}
for f_name in ['ar_preds', 'arima_preds', 'linreg_preds', 'rfreg_preds']:
    print(f'----------{f_name}-----------')
    baseline_results_full[f_name] = capstone_data_prep(f_name, 'prediction', '../data/ticker_data')
    avg_analysis_dict = {'BuyAndHold':
                     {'return_avg':[], 'sharpe_avg':[], 'max_drawdown_avg':[]},
                     'BasicStrategy':
                     {'return_avg':[], 'sharpe_avg':[], 'max_drawdown_avg':[]} }
    # get values
    for ticker, ticker_dict in baseline_results_full[f_name].items():
        for strat_name in ['BuyAndHold', 'BasicStrategy']:
            for analysis_type, value in ticker_dict[strat_name].items():
                if analysis_type == 'drawdown':
                     avg_analysis_dict[strat_name]['max_drawdown_avg'].append(value['max']['drawdown'])
                elif analysis_type == 'monthly_sharpe':
                    avg_analysis_dict[strat_name]['sharpe_avg'].append(value['sharperatio'])
                elif analysis_type == 'total_return':
                    avg_analysis_dict[strat_name]['return_avg'].append(value)

    # calculate averages
    for strat, val_dict in avg_analysis_dict.items():
        for val_type, val_list in avg_analysis_dict[strat].items():
            el_sum = 0
            for el in val_list:
                el_sum+=el
            avg_analysis_dict[strat][val_type] = el_sum / len(val_list)
    
    baseline_results_avg[f_name] = avg_analysis_dict

----------ar_preds-----------
--------Ticker Name-------: ADSK
2019-09-20, BUY EXECUTED, 153.14
Stop price: 329.07000732421875
ROI:        114.71%
***The limit sells for this strategy are valid for 1 day***
2019-09-19, Current Portfolio Value : 100000.0
2019-09-19, MARKET BUY CREATE 652 shares at next open, current close price: 153.3800048828125
2019-09-20, BUY EXECUTED, 153.14
2019-09-20, Current Portfolio Value : 100130.3980102539
2019-09-20, LIMIT SELL CREATE 652 shares at 155.29090367877183
2019-09-23, Current Portfolio Value : 99497.95721435547
2019-09-23, LIMIT SELL CREATE 652 shares at 154.30854587438162
2019-09-24, Current Portfolio Value : 97385.48358154297
2019-09-24, LIMIT SELL CREATE 652 shares at 151.02733424421638
2019-09-25, Current Portfolio Value : 96277.0756225586
2019-09-25, LIMIT SELL CREATE 652 shares at 149.30569337017928
2019-09-26, SELL EXECUTED, 149.31
2019-09-26, Current Portfolio Value : 97500.03247530611
2019-09-26, MARKET BUY CREATE 652 shares at next open,

2020-07-31, Current Portfolio Value : 152144.69730600566
2020-07-31, LIMIT SELL CREATE 643 shares at 239.4380095208472
2020-08-03, Current Portfolio Value : 154105.8492682859
2020-08-03, LIMIT SELL CREATE 643 shares at 242.52681670423547
2020-08-04, Current Portfolio Value : 154375.9080909178
2020-08-04, LIMIT SELL CREATE 643 shares at 242.9521583643039
2020-08-05, SELL EXECUTED, 242.95
2020-08-05, Current Portfolio Value : 156338.4498437257
2020-08-05, MARKET BUY CREATE 641 shares at next open, current close price: 243.1999969482422
2020-08-06, BUY EXECUTED, 243.64
2020-08-06, Current Portfolio Value : 156293.58492978528
2020-08-06, LIMIT SELL CREATE 641 shares at 246.66886410077328
2020-08-07, Current Portfolio Value : 151595.05375607926
2020-08-07, LIMIT SELL CREATE 641 shares at 239.24560519715922
2020-08-10, Current Portfolio Value : 148800.29336484388
2020-08-10, LIMIT SELL CREATE 641 shares at 234.83013381031262
2020-08-11, Current Portfolio Value : 146819.5959313722
2020-08-11,

2021-02-24, MARKET BUY CREATE 490 shares at next open, current close price: 298.9100036621094
2021-02-25, BUY EXECUTED, 294.81
2021-02-25, Current Portfolio Value : 139354.5351040179
2021-02-25, LIMIT SELL CREATE 490 shares at 287.64361400282644
2021-02-26, SELL EXECUTED, 287.64
2021-02-26, Current Portfolio Value : 141125.2065635474
2021-02-26, MARKET BUY CREATE 504 shares at next open, current close price: 276.0
2021-03-01, BUY EXECUTED, 279.86
2021-03-01, Current Portfolio Value : 143347.84840925052
2021-03-01, LIMIT SELL CREATE 504 shares at 287.8866575497676
2021-03-02, Current Portfolio Value : 140147.46071393805
2021-03-02, LIMIT SELL CREATE 504 shares at 281.455893425489
2021-03-03, Current Portfolio Value : 135802.96779401615
2021-03-03, LIMIT SELL CREATE 504 shares at 272.72619819245955
2021-03-04, SELL EXECUTED, 272.73
2021-03-04, Current Portfolio Value : 137529.7778353595
2021-03-04, MARKET BUY CREATE 508 shares at next open, current close price: 265.44000244140625
2021-03

--------Ticker Name-------: HD
2019-09-20, BUY EXECUTED, 227.99
Stop price: 368.5899963378906
ROI:        61.58%
***The limit sells for this strategy are valid for 1 day***
2019-09-19, Current Portfolio Value : 100000.0
2019-09-19, MARKET BUY CREATE 438 shares at next open, current close price: 228.1199951171875
2019-09-20, BUY EXECUTED, 227.99
2019-09-20, Current Portfolio Value : 98545.83679199217
2019-09-20, LIMIT SELL CREATE 438 shares at 225.98967556646312
2019-09-23, Current Portfolio Value : 99237.87759399413
2019-09-23, LIMIT SELL CREATE 438 shares at 227.5700246321389
2019-09-24, SELL EXECUTED, 228.14
2019-09-24, Current Portfolio Value : 100065.69732666014
2019-09-24, MARKET BUY CREATE 441 shares at next open, current close price: 225.41000366210938
2019-09-25, BUY EXECUTED, 226.59
2019-09-25, Current Portfolio Value : 100731.61163330075
2019-09-25, LIMIT SELL CREATE 441 shares at 229.4302969381704
2019-09-26, SELL EXECUTED, 229.68
2019-09-26, Current Portfolio Value : 101428

2020-07-10, Current Portfolio Value : 108991.06035162654
2020-07-10, MARKET BUY CREATE 432 shares at next open, current close price: 250.1100006103516
2020-07-13, BUY EXECUTED, 251.92
2020-07-13, Current Portfolio Value : 107997.45903326717
2020-07-13, LIMIT SELL CREATE 432 shares at 251.0757898399905
2020-07-14, SELL EXECUTED, 251.08
2020-07-14, Current Portfolio Value : 108626.36235351807
2020-07-14, MARKET BUY CREATE 417 shares at next open, current close price: 257.79000854492193
2020-07-15, BUY EXECUTED, 260.14
2020-07-15, Current Portfolio Value : 107650.5711547876
2020-07-15, LIMIT SELL CREATE 417 shares at 259.3034885906171
2020-07-16, SELL EXECUTED, 259.30
2020-07-16, Current Portfolio Value : 108277.53098740696
2020-07-16, MARKET BUY CREATE 416 shares at next open, current close price: 258.07998657226557
2020-07-17, BUY EXECUTED, 260.03
2020-07-17, Current Portfolio Value : 108423.13352646944
2020-07-17, LIMIT SELL CREATE 416 shares at 261.89855245290545
2020-07-20, Current P

...

2021-07-01, SELL EXECUTED, 64.33
2021-07-01, Current Portfolio Value : 100306.72476441463
2021-07-01, MARKET BUY CREATE 1591 shares at next open, current close price: 63.2599983215332
2021-07-02, BUY EXECUTED, 63.01
2021-07-02, Current Portfolio Value : 100561.28452164729
2021-07-02, LIMIT SELL CREATE 1591 shares at 63.94307782210333
2021-07-06, Current Portfolio Value : 97697.48573548398
2021-07-06, LIMIT SELL CREATE 1591 shares at 63.77181417853584
2021-07-07, Current Portfolio Value : 96170.12719208797
2021-07-07, LIMIT SELL CREATE 1591 shares at 62.23703238485697
2021-07-08, Current Portfolio Value : 95740.55646378596
2021-07-08, LIMIT SELL CREATE 1591 shares at 61.459799803539156
2021-07-09, Current Portfolio Value : 97474.7467065533
2021-07-09, MARKET SELL CREATE. PREDICTION < CURRENT CLOSE
2021-07-12, SELL EXECUTED, 60.42
2021-07-12, Current Portfolio Value : 96186.03452164729
2021-07-12, MARKET BUY CREATE 1576 shares at next open, current close price: 61.16999816894531
2021-07-

--------Ticker Name-------: ADSK
2019-09-19, BUY EXECUTED, 151.39
Stop price: 329.07000732421875
ROI:        117.27%
***The limit sells for this strategy are valid for 1 day***
2019-09-18, Current Portfolio Value : 100000.0
2019-09-18, MARKET BUY CREATE 660 shares at next open, current close price: 151.11000061035156
2019-09-19, BUY EXECUTED, 151.39
2019-09-19, Current Portfolio Value : 101313.40362548828
2019-09-19, LIMIT SELL CREATE 660 shares at 155.37790990246262
2019-09-20, Current Portfolio Value : 101286.99798583984
2019-09-20, LIMIT SELL CREATE 660 shares at 155.32358475364103
2019-09-23, Current Portfolio Value : 100646.79718017578
2019-09-23, LIMIT SELL CREATE 660 shares at 154.34324402170625
2019-09-24, Current Portfolio Value : 98508.40362548828
2019-09-24, LIMIT SELL CREATE 660 shares at 151.0777343031669
2019-09-25, Current Portfolio Value : 97386.39556884766
2019-09-25, LIMIT SELL CREATE 660 shares at 149.37878765746945
2019-09-26, SELL EXECUTED, 149.38
2019-09-26, Curre

2020-08-12, SELL EXECUTED, 231.86
2020-08-12, Current Portfolio Value : 141211.0997361784
2020-08-12, MARKET BUY CREATE 606 shares at next open, current close price: 231.57000732421875
2020-08-13, BUY EXECUTED, 232.88
2020-08-13, Current Portfolio Value : 144428.95825668622
2020-08-13, LIMIT SELL CREATE 606 shares at 241.258999522569
2020-08-14, Current Portfolio Value : 142101.92047592447
2020-08-14, LIMIT SELL CREATE 606 shares at 237.38356779679944
2020-08-17, Current Portfolio Value : 144919.81677719404
2020-08-17, LIMIT SELL CREATE 606 shares at 242.04901613615897
2020-08-18, SELL EXECUTED, 242.05
2020-08-18, Current Portfolio Value : 146767.52055570637
2020-08-18, MARKET BUY CREATE 600 shares at next open, current close price: 242.69000244140625
2020-08-19, BUY EXECUTED, 244.40
2020-08-19, Current Portfolio Value : 144979.5231191829
2020-08-19, LIMIT SELL CREATE 600 shares at 244.39243789232216
2020-08-20, SELL EXECUTED, 244.39
2020-08-20, Current Portfolio Value : 146762.9869532

2021-06-29, SELL EXECUTED, 296.85
2021-06-29, Current Portfolio Value : 150027.71301872455
2021-06-29, MARKET BUY CREATE 505 shares at next open, current close price: 297.739990234375
2021-06-30, BUY EXECUTED, 296.67
2021-06-30, Current Portfolio Value : 147618.8531554433
2021-06-30, LIMIT SELL CREATE 505 shares at 295.4882799564665
2021-07-01, Current Portfolio Value : 148527.86240226947
2021-07-01, LIMIT SELL CREATE 505 shares at 297.2963250476329
2021-07-02, SELL EXECUTED, 297.30
2021-07-02, Current Portfolio Value : 150344.0003867733
2021-07-02, MARKET BUY CREATE 504 shares at next open, current close price: 297.739990234375
2021-07-06, BUY EXECUTED, 298.19
2021-07-06, Current Portfolio Value : 149038.6422324764
2021-07-06, LIMIT SELL CREATE 504 shares at 299.2032535897995
2021-07-07, SELL EXECUTED, 299.20
2021-07-07, Current Portfolio Value : 150854.67896556348
2021-07-07, MARKET BUY CREATE 516 shares at next open, current close price: 296.1099853515625
2021-07-08, BUY EXECUTED, 2

--------Ticker Name-------: HD
2019-09-19, BUY EXECUTED, 231.15
Stop price: 368.5899963378906
ROI:        59.37%
***The limit sells for this strategy are valid for 1 day***
2019-09-18, Current Portfolio Value : 100000.0
2019-09-18, MARKET BUY CREATE 432 shares at next open, current close price: 230.8300018310547
2019-09-19, BUY EXECUTED, 231.15
2019-09-19, Current Portfolio Value : 98691.04052734374
2019-09-19, LIMIT SELL CREATE 432 shares at 230.00485633306045
2019-09-20, SELL EXECUTED, 230.00
2019-09-20, Current Portfolio Value : 99505.30057260086
2019-09-20, MARKET BUY CREATE 443 shares at next open, current close price: 224.6699981689453
2019-09-23, BUY EXECUTED, 224.16
2019-09-23, Current Portfolio Value : 100431.1689502864
2019-09-23, LIMIT SELL CREATE 443 shares at 228.16667070100053
2019-09-24, SELL EXECUTED, 228.17
2019-09-24, Current Portfolio Value : 101280.25407082963
2019-09-24, MARKET BUY CREATE 446 shares at next open, current close price: 225.41000366210938
2019-09-25, 

### LSTM Results

In [ ]:
lstm_results_full = capstone_data_prep('LSTM_preds', 'prediction', '../data/ticker_data')

avg_analysis_dict_lstm = {'BuyAndHold':
                     {'return_avg':[], 'sharpe_avg':[], 'max_drawdown_avg':[]},
                     'BasicStrategy':
                     {'return_avg':[], 'sharpe_avg':[], 'max_drawdown_avg':[]} }
# get values
for ticker, ticker_dict in lstm_results_full.items():
    for strat_name in ['BuyAndHold', 'BasicStrategy']:
        for analysis_type, value in ticker_dict[strat_name].items():
            if analysis_type == 'drawdown':
                 avg_analysis_dict_lstm[strat_name]['max_drawdown_avg'].append(value['max']['drawdown'])
            elif analysis_type == 'monthly_sharpe':
                avg_analysis_dict_lstm[strat_name]['sharpe_avg'].append(value['sharperatio'])
            elif analysis_type == 'total_return':
                avg_analysis_dict_lstm[strat_name]['return_avg'].append(value)

# calculate averages
for strat, val_dict in avg_analysis_dict_lstm.items():
    for val_type, val_list in avg_analysis_dict_lstm[strat].items():
        el_sum = 0
        for el in val_list:
            el_sum+=el
        avg_analysis_dict_lstm[strat][val_type] = el_sum / len(val_list)

## Plotting backtest results

In [ ]:
import matplotlib.pyplot as plt

### Buy and Hold for the entire trading period

#### Average buy and hold vs avg buy and hold per model

In [ ]:
buy_hold_avg = avg_analysis_dict_xgb['BuyAndHold']
avg_per_model_df=pd.DataFrame(buy_hold_avg, index={'BuyAndHold':0})

In [ ]:
avg_per_model_df = avg_per_model_df.append(pd.DataFrame(avg_analysis_dict_xgb['BasicStrategy'], index={'XGB':0}))
avg_per_model_df = avg_per_model_df.append(pd.DataFrame(avg_analysis_dict_lstm['BasicStrategy'], index={'LSTM':0}))
for model in baseline_results_avg.keys():
    avg_per_model_df = avg_per_model_df.append(pd.DataFrame(baseline_results_avg[model]['BasicStrategy'], index={model:0}))

In [ ]:
avg_per_model_df

In [ ]:
avg_per_model_df['max_drawdown_avg'] = avg_per_model_df['max_drawdown_avg'] / 100
avg_per_model_df.index = ['BuyAndHold', 'XGB', 'LSTM', 'AR', 'ARIMA', 'LineaRegression', 'RandomForest']

In [ ]:
plt.style.use('fivethirtyeight')
fig, ax = plt.subplots(figsize=(8,5))
ax.set_title('Model Comparison of Ticker-Averaged Results Over 544 Trading Days', fontsize=12)
avg_per_model_df.plot(kind='bar', ax=ax)

#### Returns per ticker per strategy

In [ ]:
return_per_ticker_df = pd.DataFrame(xgb_backtesting_dict)
return_per_ticker_df = return_per_ticker_df.append(pd.DataFrame(lstm_results_full).loc['BasicStrategy', :])

In [ ]:
for model in baseline_results_avg.keys():
    return_per_ticker_df = return_per_ticker_df.append(pd.DataFrame(baseline_results_full[model]).loc['BasicStrategy', :])

In [ ]:
return_per_ticker_df.index = ['BuyAndHold', 'XGB', 'LSTM', 'AR', 'ARIMA', 'LineaRegression', 'RandomForest']

In [ ]:
# this holds all results per ticker per model in a dataframe, we just want returns for now
total_result_df = return_per_ticker_df.copy()
total_result_df

In [ ]:
for idx in  return_per_ticker_df.index:
    for column in  return_per_ticker_df.columns:
         return_per_ticker_df.loc[idx, column] = return_per_ticker_df.loc[idx, column]['total_return']

In [ ]:
return_per_ticker_df = return_per_ticker_df.apply(lambda x: x*100, axis=0)

In [ ]:
plt.style.use('fivethirtyeight')
fig, ax = plt.subplots(figsize=(20,8))
ax.set_ylabel('% Return from $100k Starting Cash')
ax.set_title('Model Comparison of Total Returns Over 544 Trading Days')
return_per_ticker_df.swapaxes("index", "columns").plot(y=list(return_per_ticker_df.index), kind='bar', ax=ax)

### Library additionally has plotting
Documentation talks about keeping statistics at the bottom
https://www.backtrader.com/docu/observers-and-statistics/observers-and-statistics/

#### Plot XGBoost vs LSTM vs Linear Regression trading for ADSK

In [ ]:
plt.style.use('default')

In [ ]:
xgb_backtesting_dict['ADSK']['BasicStrategy']['cerebro_object'].plot()

In [ ]:
# plt.rcParams['figure.figsize'] = [10, 8]
# plt.rcParams.update({'font.size': 12})
lstm_results_full['ADSK']['BasicStrategy']['cerebro_object'].plot()

In [ ]:
baseline_results_full['linreg_preds']['ADSK']['BasicStrategy']['cerebro_object'].plot()

### Ways to Improve:

Increase strategy trading volume. There are certain periods where sell orders are not going through for multiple days in a row, meaning that the price is lower than anticipated by model for that period. For example, if the last sell order didn't go through, we can find a way to use that information to inform our next sell order and make the prediction lower or try a market sell. Maybe train a "bear" model, one that only knows pain and suffering of bear markets, and only produces pessimistic high price predictions.   

Smarter entry points. Currently, buy orders are being executed at market price at the open (getting in position ASAP when not in a position). Returns may increase if entry constraints are increased.  

Resizing orders. Orders for this strategy baseline are currently 100% in or 100% out of position. This has beneifits if you want less exposure to the long term movement of the stocks. Depending on the situation and strategy, we might want to reserve a cash position to expose ourselves less to short term movements. 

Model ensembling. Models with similar R^2 scores on the test data can be ensembled together to create better prediction results.